In [1]:
print("Testing......")

Testing......


In [2]:
!pip install transformers datasets peft accelerate -U bitsandbytes
!pip install fastapi uvicorn python-multipart pyngrok

^C
  Using cached PyYAML-6.0.2-cp313-cp313-win_amd64.whl.metadata (2.1 kB)

   ---------------------------------------- 0/3 [PyYAML]
   ---------------------------------------- 0/3 [PyYAML]
   ---------------------------------------- 0/3 [PyYAML]
   ---------------------------------------- 0/3 [PyYAML]
   ------------- -------------------------- 1/3 [python-multipart]
   ------------- -------------------------- 1/3 [python-multipart]
   -------------------------- ------------- 2/3 [pyngrok]
   -------------------------- ------------- 2/3 [pyngrok]
   -------------------------- ------------- 2/3 [pyngrok]
   -------------------------- ------------- 2/3 [pyngrok]
   -------------------------- ------------- 2/3 [pyngrok]
   -------------------------- ------------- 2/3 [pyngrok]
   -------------------------- ------------- 2/3 [pyngrok]
   ---------------------------------------- 3/3 [pyngrok]



In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "microsoft/phi-2"

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit = True,
    device_map = "auto"
)

print("Tokenizer and model loaded")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Tokenizer and model loaded


In [ ]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType

model = prepare_model_for_kbit_training(model)

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj","v_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(model,lora_config)

model.print_trainable_parameters()

trainable params: 2,621,440 || all params: 2,782,305,280 || trainable%: 0.0942


In [ ]:
import json

with open("Phi2_finetune\dataset.json","r") as f:
  data = json.load(f)

print(json.dumps(data[:1],indent=5))

[
     {
          "prompt": "Why do snakes stick out their tongues?",
          "response": "Snakes use their tongues to smell the air and find food or sense danger around them."
     }
]


In [ ]:
train_texts = [
    f"### Prompt:\n{item['prompt']}\n\n### Response:\n{item['response']}"
    for item in data
]

print(train_texts[0])

### Prompt:
Why do snakes stick out their tongues?

### Response:
Snakes use their tongues to smell the air and find food or sense danger around them.


In [ ]:
from datasets import Dataset
from transformers import AutoTokenizer

dataset = Dataset.from_dict({"text":train_texts})

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

def tokenize(example):
  return tokenizer(example["text"],truncation=True,padding="max_length",max_length=512)

tokenized_dataset = dataset.map(tokenize,batched=True)

print(tokenized_dataset[0])

Map:   0%|          | 0/320 [00:00<?, ? examples/s]

{'text': '### Prompt:\nWhy do snakes stick out their tongues?\n\n### Response:\nSnakes use their tongues to smell the air and find food or sense danger around them.', 'input_ids': [21017, 45965, 25, 198, 5195, 466, 26042, 4859, 503, 511, 39413, 30, 198, 198, 21017, 18261, 25, 198, 16501, 1124, 779, 511, 39413, 284, 8508, 262, 1633, 290, 1064, 2057, 393, 2565, 3514, 1088, 606, 13, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 5

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./lora_finetuned_model",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    learning_rate=2e-4,
    num_train_epochs=3,
    logging_steps=10,
    save_steps=100,
    save_total_limit=2,
    report_to="none",
    fp16=True,
    push_to_hub=False
)

In [ ]:
from transformers import Trainer, DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator
)

/tmp/ipython-input-9-1550599017.py:7: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/bitsandbytes/nn/modules.py:463: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we wi

Step,Training Loss
10,2.300600
20,1.445800
30,0.804800
40,0.577300
50,0.434400
60,0.361400
70,0.322100
80,0.255500
90,0.244300
100,0.265700


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


TrainOutput(global_step=120, training_loss=0.6289072771867116, metrics={'train_runtime': 726.6739, 'train_samples_per_second': 1.321, 'train_steps_per_second': 0.165, 'total_flos': 7818805090713600.0, 'train_loss': 0.6289072771867116, 'epoch': 3.0})

In [ ]:
### Testing with Manual inputs

from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
import torch
import gc

# Load model and tokenizer
model_name = "microsoft/phi-2"
checkpoint_path = "/content/lora_finetuned_model/checkpoint-120"

base_model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = PeftModel.from_pretrained(base_model, checkpoint_path, device_map="auto")
model.eval()

# Generate
prompt = "### Prompt:\nWhat is the color of sky?\n\n### Response:\n"
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

with torch.no_grad():
    outputs = model.generate(**inputs, max_new_tokens=100)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

#  Clear memory (GPU and CPU)
del model
del base_model
del tokenizer
del inputs
del outputs

gc.collect()              # Free Python garbage
torch.cuda.empty_cache()  # Free GPU memory
torch.cuda.ipc_collect()  # Reclaim inter-process GPU memory (optional)


In [ ]:
!pip install pyngrok fastapi uvicorn

In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

import torch
import gc  # for RAM garbage collection

app = FastAPI()

# ---- Model configuration ----
model_name = "microsoft/phi-2"
checkpoint_dir = "/content/lora_finetuned_model/checkpoint-120"

# ---- Request body schema ----
class QueryRequest(BaseModel):
    userQuestion: str

@app.post("/generate")
def generate_response(request: QueryRequest):
    # Load model and tokenizer only inside the route
    base_model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = PeftModel.from_pretrained(base_model, checkpoint_dir, device_map="auto")
    model.eval()

    prompt = f"### Prompt:\n{request.userQuestion}\n\n### Response:\n"
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=100)

    result = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # 🧹 Cleanup
    del model
    del base_model
    del tokenizer
    del inputs
    del outputs

    gc.collect()
    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()

    return {"response": result}

In [ ]:
import uvicorn
import nest_asyncio
from pyngrok import ngrok

nest_asyncio.apply()

public_url = ngrok.connect(8000)
print("Your API is live at:", public_url)

uvicorn.run(app, port=8000)


Your API is live at: NgrokTunnel: "https://a650-34-125-232-50.ngrok-free.app" -> "http://localhost:8000"


INFO:     Started server process [421]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-6' coro=<Server.serve() done, defined at /usr/local/lib/python3.11/dist-packages/uvicorn/server.py:68> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/main.py", line 580, in run
    server.run()
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/server.py", line 66, in run
    return asyncio.run(self.serve(sockets=sockets))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 92, in run_until_complete
    self._run_once()
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 133, in _run_once
    handle._run()

INFO:     110.172.151.100:0 - "POST /generate HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [421]
